In [66]:
// SETUP
import java.io.File
import java.io.BufferedReader

fun readInputLines(dayIdx: Int): Iterable<String> {
    val fileName = "../data/%02d.txt".format(dayIdx)
    val reader: BufferedReader = File(fileName).bufferedReader()
    return Iterable { reader.lineSequence().iterator() }
}

fun readInputLines(dayIdx: Int, testIdx: Int): Iterable<String> {
    val fileName = "../data/%02dt%d.txt".format(dayIdx, testIdx)
    val reader: BufferedReader = File(fileName).bufferedReader()
    return Iterable { reader.lineSequence().iterator() }
}

In [67]:
val nums = readInputLines(1).map {it.toInt()}

fun countIncr(nums: Iterable<Int>) = nums.windowed(2).filter {(a, b) -> a < b}.count()

val res1 = countIncr(nums)
val res2 = countIncr(nums.windowed(3).map {it.sum()})

println("Answer 1: $res1\nAnswer 2: $res2")

Answer 1: 1557
Answer 2: 1608


In [68]:
enum class Command {
  forward, up, down
}

fun parseLine(line: String): Pair<Command, Int> { 
    val (c, n) = line.split(" ")
    return Pair(Command.valueOf(c), n.toInt())
}

fun step1(pos: Pair<Int, Int>, cmdLine: Pair<Command, Int>): Pair<Int, Int> {
    val (x, depth) = pos
    val (cmd, offs) = cmdLine
    return when (cmd) {
        Command.up -> Pair(x, depth - offs)
        Command.down -> Pair(x, depth + offs)
        Command.forward -> Pair(x + offs, depth)
    }
}

fun step2(pos: Triple<Int, Int, Int>, cmdLine: Pair<Command, Int>): Triple<Int, Int, Int> {
    val (x, depth, aim) = pos
    val (cmd, offs) = cmdLine
    return when (cmd) {
        Command.up -> Triple(x, depth, aim - offs)
        Command.down -> Triple(x, depth, aim + offs)
        Command.forward -> Triple(x + offs, depth + aim * offs, aim)
    }
}

val commands = readInputLines(2).map {parseLine(it)}
val (a1, b1) = commands.fold(Pair(0, 0), ::step1)
val (a2, b2) = commands.fold(Triple(0, 0, 0), ::step2)
println("Answer 1: ${a1 * b1}\nAnswer 2: ${a2 * b2}")

Answer 1: 1728414
Answer 2: 1765720035
